<a href="https://colab.research.google.com/github/mohammad-rahbari/Federated-Learning-MLDL/blob/master/notebooks/MLDL_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import pandas as pd
import numpy as np

# Extract trained setting

In [3]:

def get_seed_models(df):
  filter = df['round_number'] == 0
  df_filtered = df[filter]
  return df_filtered

def get_model_squence(df, seed_model):
  result = df[df['model_name']==seed_model['model_name']]
  latest_model = result['model_name'].values[0]
  while True:
    if latest_model in df['prev_global_model_name'].values:
      temp =  df[df['prev_global_model_name']==latest_model]
      latest_model = temp['model_name'].values[0]
      result = pd.concat([result, temp])
    else:
      break
  result = result.sort_values(by=['round_number'])
  return result

def get_from_client(df_client, model_name, attribute, single_record= True):
  if single_record:
    return df_client[df_client['initial_model_name']==model_name][attribute].values[0]
  else:
    return df_client[df_client['initial_model_name']==model_name][attribute]

def get_vlid_lst(lst):
  result = []
  for i in lst:

    if not (i is None or np.isnan(i)):
      result.append(i)

  return result

In [ ]:
df_global = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
df_client = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")

seed_models = get_seed_models(df_global)
for i in range(len(seed_models)):

  log = f"{i+1})\n"


  df_seq = get_model_squence(df_global,seed_models.iloc[i] )
  r_num = max(get_vlid_lst(df_seq['round_number'].values))
  if r_num != 0:

    log +=f"Seed model: {seed_models.iloc[i]['model_name']} \n"
    log +=f"Spliting method: {df_seq['splitting_method'].values[-1]}  \n"
    if df_seq['splitting_method'].values[-1] == "non-i.i.d. sharing":
      log +=f"Nc: {max(get_vlid_lst(df_seq['Nc'].values))}  \n"
    log +=f"Number of rounds: {r_num}  \n"
    log +=f"Number of local_steps: {max(get_vlid_lst(df_seq['num_local_steps'].values))}  \n"
    try:
      log +=f"Target Sparcity: {get_from_client(df_client, seed_models.iloc[i]['model_name'], 'sparsity')}  \n"
    except:
      f"Target Sparcity: {0.5}  \n"
      pass
    log +=f"Accuracy: {df_seq[df_seq['round_number']== r_num]['accuracy'].values[-1]}  \n"
    log +=f"Loss: {df_seq[df_seq['round_number']== r_num]['loss'].values[-1]} \n"


    print(log," \n \n \n")







In [32]:
modename_targetround= {
    "3b16bcde-abf5-4c87-b9bb-ef2827a59c6d":25,
    "1f7efd38-b55a-458c-9478-b763895e5acc":25,
    "165b689b-6b5e-48eb-9d24-ae343a039013":25,
    "0af55aa3-0139-49aa-95c4-854a511ad30d":25,
    "3fba6992-552f-420f-877f-82afd3f0ef6f":50,
    "f77e86b0-b0e3-4cd8-b65a-40323e81b49b":50,
    "fdb2734f-a092-4641-a843-969dcc993937":100,
    "91be5aaa-6416-4558-89d4-a8c18e6d1e2f":50,
    "0decec9d-3280-48d6-8c4e-ad1785b9736e":50,
    "f60e06e4-ac4b-4495-8904-16753c21c425":25,
    "453429aa-5c7f-4046-b4f2-658588fc9c38":25,
    "201a5aeb-0d1e-438f-8178-4ebcc4d94435":50,
    "23f056c6-52e9-47b2-beb1-9ef590f743cc":100,
    "61cb31ef-74c1-4f60-95e9-d35fd60a7d55":100,
    "f56d4e15-9536-4733-9120-6423230a650d":100,
    "f142f936-1ed1-4dfb-b0b2-3f99aff46a89":100,
    "612d19a1-3b97-466f-a17f-892ce36bd5c9":25
    }
data_for_new_df = []

df_global = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/global_log.csv")
df_client = pd.read_csv("/content/drive/MyDrive/MLDL_FederatedLearning/csv/client_log.csv")



seed_models = get_seed_models(df_global)
for i in range(len(seed_models)):
  df_seq = get_model_squence(df_global,seed_models.iloc[i] )
  seed_name = seed_models.iloc[i]['model_name']
  if seed_name not in modename_targetround.keys():
    continue
  record = df_seq[df_seq["round_number"]==modename_targetround[seed_name]]
  sparsity =  get_from_client(df_client, seed_models.iloc[i]['model_name'], 'sparsity') if seed_name != "612d19a1-3b97-466f-a17f-892ce36bd5c9" else 0.5



  data_for_new_df.append({

        'round_number': record['round_number'].values[0],
        'local_step': record['num_local_steps'].values[0],
        'sparsity': sparsity,
        'Nc': record['Nc'].values[0],
        'accuracy': record['accuracy'].values[0],
        'loss': record['loss'].values[0],
        'aggregation_method' : record["aggregation_method"].values[0],
        'spliting_method': record['splitting_method'].values[0],
        'seed_model_name': seed_name,
        'last_model_name': record["model_name"].values[0]

    })


result_df = pd.DataFrame(data_for_new_df)


display(result_df)

,round_number,local_step,sparsity,Nc,accuracy,loss,aggregation_method,spliting_method,seed_model_name,last_model_name
0,100,4.0,0.5,100.0,37.55,2.509509,FedAvg,i.i.d. sharing,f142f936-1ed1-4dfb-b0b2-3f99aff46a89,900f32b7-9a03-4eaf-b252-feaa8944678d
1,100,4.0,0.5,50.0,42.89,2.220242,FedAvg,non-i.i.d. sharing,f56d4e15-9536-4733-9120-6423230a650d,6f243790-ba8b-4bf0-8fdc-15ca06b78067
2,100,4.0,0.5,10.0,42.20,2.311263,FedAvg,non-i.i.d. sharing,61cb31ef-74c1-4f60-95e9-d35fd60a7d55,d061c76f-18ab-4e8b-b2df-7392c9cdb8c6
3,100,0.0,0.5,0.0,36.99,2.483904,FedAvg,non-i.i.d. sharing,23f056c6-52e9-47b2-beb1-9ef590f743cc,766ea6e6-f387-4939-b06f-a92141f40947
4,50,0.0,0.5,0.0,47.13,2.006864,FedAvg,i.i.d. sharing,201a5aeb-0d1e-438f-8178-4ebcc4d94435,ba641ff5-2a41-49f9-bd49-07b0a9a595a3
5,25,16.0,0.5,100.0,54.13,1.704356,FedAvg,i.i.d. sharing,612d19a1-3b97-466f-a17f-892ce36bd5c9,96e69034-1f33-4cb4-af98-5fedc9223e2d
6,25,16.0,0.5,50.0,55.72,1.633388,FedAvg,non-i.i.d. sharing,453429aa-5c7f-4046-b4f2-658588fc9c38,afda943f-f16c-432a-ac3f-049d507f9ba3
7,25,16.0,0.5,1.0,9.38,6.838218,FedAvg,non-i.i.d. sharing,f60e06e4-ac4b-4495-8904-16753c21c425,81667e00-23ee-4ff7-965d-2965bd0fb750
8,50,8.0,0.5,50.0,51.73,1.817291,FedAvg,non-i.i.d. sharing,0decec9d-3280-48d6-8c4e-ad1785b9736e,94781c1d-c50e-4ec9-afd2-358a0f6850c6
9,50,8.0,0.5,10.0,45.61,2.107623,FedAvg,non-i.i.d. sharing,91be5aaa-6416-4558-89d4-a8c18e6d1e2f,f48789e6-5511-4885-b57f-563bccb93d2b


In [ ]:
# Create an empty list to store the data for the new DataFrame
data_for_new_df = []

# Get the seed models
seed_models = get_seed_models(df_global)

# Iterate through the seed models
for i in range(len(seed_models)):
    seed_model_info = seed_models.iloc[i]
    seed_model_name = seed_model_info['model_name']

    # Get the model sequence for the current seed model
    df_seq = get_model_squence(df_global, seed_model_info)

    # Get the last model name in the sequence
    last_model_name = df_seq['model_name'].values[-1]

    # Get the splitting method
    splitting_method = df_seq['splitting_method'].values[-1]

    # Get the round number
    round_number = max(get_vlid_lst(df_seq['round_number'].values))

    # Get the number of local steps
    local_step = max(get_vlid_lst(df_seq['num_local_steps'].values))

    # Get the sparsity from df_client
    try:
        sparsity = get_from_client(df_client, seed_model_name, 'sparsity')
    except:
        sparsity = 0.5  # Default value if sparsity is not found

    # Get Nc, accuracy, and loss from the last round
    if round_number != 0:
        last_round_data = df_seq[df_seq['round_number'] == round_number]
        Nc = max(get_vlid_lst(last_round_data['Nc'].values)) if splitting_method == "non-i.i.d. sharing" else None
        accuracy = last_round_data['accuracy'].values[-1]
        loss = last_round_data['loss'].values[-1]
    else:
        Nc = None
        accuracy = None
        loss = None

    # Append the extracted data to the list
    data_for_new_df.append({
        'seed_model_name': seed_model_name,
        'last_model_name': last_model_name,
        'spliting_method': splitting_method,
        'round_number': round_number,
        'local_step': local_step,
        'sparsity': sparsity,
        'Nc': Nc,
        'accuracy': accuracy,
        'loss': loss
    })

# Create the new DataFrame
result_df = pandas.DataFrame(data_for_new_df)

# Display the new DataFrame
display(result_df)